# 快速入门

PyTorch 有两个用于处理数据的原语 ： torch.utils.data.DataLoader 和 torch.utils.data.Dataset 的 Dataset。 
Dataset 存储示例及其相应的标签，DataLoader 围绕 Dataset 包装一个可迭代对象。

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch 提供特定于域的库，例如 TorchText、 TorchVision 和 TorchAudio，它们都包含数据集。在本教程中，我们将使用 TorchVision 数据集。

torchvision.datasets 模块包含许多真实世界视觉数据（如 CIFAR、COCO）的数据集对象。在本教程中，我们使用 FashionMNIST 数据集。
每个 TorchVision 数据集都包含两个参数：transform 和 target_transform 分别修改样品和标签。

In [12]:
# Download training data from open datasets.
# 从开放数据集下载训练数据
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor(),
    
)
# 从开放数据集下载测试数据
test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor(),
)

100%|█████████████████████████████████████████████████████████████████████████████| 26.4M/26.4M [15:27<00:00, 28.5kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 57.3kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 4.42M/4.42M [01:35<00:00, 46.4kB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████| 5.15k/5.15k [00:00<?, ?B/s]


我们将 Dataset 作为参数传递给 DataLoader。这会在我们的数据集上包装一个可迭代对象，并支持自动批处理、采样、打乱和多进程数据加载。
在这里，我们定义了 64 的批量大小，即 dataloader 可迭代对象中的每个元素将返回一批 64 个特征和标签。

In [23]:
batch_size = 64

# 创建 data loader
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f'Shape of X[N, C, H, W]:{X.shape}')
    print(f'Shape of y:{y.shape} {y.dtype}')
    break



Shape of X[N, C, H, W]:torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64


为了在 PyTorch 中定义神经网络，创建一个继承自__nn__的类  。在 __init__ 函数中定义网络层，并在正向函数中指定数据将如何通过网络。
为了加速神经网络中的作，将其移动到加速器,例如 CUDA、MPS、MTIA 或 XPU。如果当前的加速器可用，使用它。否则，使用 CPU。

In [16]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f'Using {device} device')

Using cpu device


In [27]:
# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    #前向传播
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)
        

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [38]:
# <!-- 优化模型参数：要训练模型，需要一个损失函数和优化器 。 -->
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [40]:
# 在单个训练循环中，模型对训练数据集进行预测（批量提供给它），并反向传播预测误差以调整模型的参数。
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        #计算误差
        pred = model(X)
        loss = loss_fn(pred, y)

        #后向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f'loss:{loss:>7f}  [{current:>5d}/{size:>5d}]')

In [42]:
# 我们还根据测试数据集检查模型的性能，以确保它正在学习。
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [44]:
# 训练过程在多次迭代（ 纪元 ）中进行。在每个时期，模型都会学习参数以做出更好的预测。
# 在每个时期打印模型的准确性和损失;我们希望看到准确性随着每个纪元的提高和损失的减少而减少。

epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n------------------------------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!!!")

Epoch 1
------------------------------------------------------
loss:2.299602  [   64/60000]
loss:2.289397  [ 6464/60000]
loss:2.265764  [12864/60000]
loss:2.277863  [19264/60000]
loss:2.248036  [25664/60000]
loss:2.220093  [32064/60000]
loss:2.230238  [38464/60000]
loss:2.194329  [44864/60000]
loss:2.186302  [51264/60000]
loss:2.164025  [57664/60000]
Test Error: 
 Accuracy: 44.6%, Avg loss: 2.154938 

Epoch 2
------------------------------------------------------
loss:2.159341  [   64/60000]
loss:2.152780  [ 6464/60000]
loss:2.091779  [12864/60000]
loss:2.117631  [19264/60000]
loss:2.053763  [25664/60000]
loss:2.001616  [32064/60000]
loss:2.027052  [38464/60000]
loss:1.953698  [44864/60000]
loss:1.949970  [51264/60000]
loss:1.882244  [57664/60000]
Test Error: 
 Accuracy: 61.5%, Avg loss: 1.879908 

Epoch 3
------------------------------------------------------
loss:1.913227  [   64/60000]
loss:1.885152  [ 6464/60000]
loss:1.761379  [12864/60000]
loss:1.804640  [19264/60000]
loss:1.6879

保存模型
保存模型的常用方法是序列化内部状态字典（包含模型参数）。

In [51]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [55]:
# 加载模型  加载模型的过程包括重新创建模型结构并将状态字典加载到其中
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [59]:
# 用该模型进行预测
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Bag", Actual: "Ankle boot"
